[Back to Overview](overview.ipynb)

# Remembering Python

Looking back on this course, you'll have had some quality time with but a few of the many riches stored out there in the cloud:  Sphinx, Django, Flask, pandas, PyCrypto, pillow, matplotlib and many more.

We also considered the many IDEs (with an emphasis on Spyder) and kept returning to Jupyter Notebooks (as we're doing here), and Jupyter Labs.  Anaconda was our default distribution.

However the main focus was on making the code readable, comprehensible, and that meant we focused on grammar and a core vocabulary.

Coming out the other end of the tunnel, it pays to review.

What were some of the core ideas again?

Our review here covers some of the basic concepts, but in the examples reaches into some of the more arcane corners of the language.  Python remains engaging and entertaining, is the message there.

## Mutable versus Immutable

As you continue to explore the many languages, such as Clojure and Rust, you will come to appreciate the big difference immutability makes.  When you can count on an object to stay constant, the compiler has so much less to worry about.  

Python's overall permissiveness and mutability, means it's a runtime experience first and foremost, with the compiling step not really a time for exhaustive checking, at least not for type consistency.  You might get some help from 3rd party tools, as Python does support type annotations.

In [1]:
def cubocta(n : int) -> int:
    """
    https://oeis.org/A005901
    
    Number of points on surface of cuboctahedron (or icosahedron): 
    a(0) = 1; for n > 0, a(n) = 10n^2 + 2. Also coordination 
    sequence for f.c.c. or A_3 or D_3 lattice. 
    """
    if n == 1:
        return 1
    return 10 * n**2 + 2

print(cubocta.__annotations__)

{'n': <class 'int'>, 'return': <class 'int'>}


When you encounter a new type, or revisit an old one, sometimes ask yourself if it's mutable and remember, if it is, it won't be suitable as a dict key.  That's not the only criterion however.  Dictionary keys must also be "hashable".  Lets explore all these concepts.

First questions: are slices immutable and might they therefore serve as dict keys?

In [2]:
bread_slices = slice(3,7)
loaf = ("slice " * 10).split()
loaf[bread_slices]

['slice', 'slice', 'slice', 'slice']

Is the slice type at all mutable?  Once upper and lower bounds are provided, along with stride or step -- the optional 3rd argument -- is there anything we might do to alter these attributes?  

If not, that would seem to make them immutable.

In [3]:
", ".join(dir(bread_slices))

'__class__, __delattr__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, indices, start, step, stop'

In [4]:
bread_slices.start

3

In [5]:
# here's the test
try:
    bread_slices.start = 4
except Exception as e:
    print(e)

readonly attribute


And yet, we'll find ```bread_slices```, immutable as it seems, is not any good as a dict key.

In [6]:
print(type(bread_slices))
the_dict = dict()
try:
    the_dict[bread_slices] = ['m','u','t','a','b','l','e',' ','v','a','l','u','e']
except:
    print("Nice try, no cigar")  # idiom meanining desired outcome not achieved, though not for lack of meritorious endeavor

<class 'slice'>
Nice try, no cigar


We've learned in the above experiment, that no, a slice is not hashable, meaning there's no way to compare its uniqueness to something else, other than by memory location.

In [7]:
bread_slices > slice(1,None)

True

In [8]:
s1 = slice(0,9)
s2 = slice(0,8)
s1 > s2

True

Aha!  A clue.  The slice type does implement the comparator operators, based on comparing the (start, stop) tuples, element by element.

Lets go back to where we have no comparitor operators whatsoever...

In [9]:
class Dog:
    pass

kennel = {Dog():"spunky", Dog():"needs shots", Dog():"pickup tomorrow"}

Dog() is Dog()

False

That worked!  

Dog instances with nothing but memory locations to tell them apart, may serve as keys. 

However as soon as you hint that dog instances are orderable, by defining `__eq__`, you're suggesting they're likewise hashable, and yet ```__hash__``` is undefined, as in we're no longer sure what "identical" means, as there's a value developing.  There's no fallback to using memeory id.

In [10]:
class Dog:
    def __init__(self, v):
        self.v = v
    def __eq__(self, other):
        return self.v == other.v

try: 
    kennel = {Dog(1):"spunky", Dog(2):"needs shots", Dog(3):"pickup tomorrow"}
except Exception as e:
    print(e)

unhashable type: 'Dog'


Lets see if we might repair our dog's suitability as a dict key, by making hashability again depend on nothing more than memory location, surely unique per each object.  

There's no longer a danger of two dog instances being equal, that hadn't been before.  This is what corrupts a dictionary, when any two keys are no longer unique.  Depending on some `v` was going down the wrong path.

When the Dog instances had no comparison capabilities, the fallback reliance on memory location was sufficient to assure hashability.

In [11]:
class Dog(object):
    pass

dog1 = Dog()
dog2 = Dog()
print(dog1 == dog2)

False


In [12]:
class Dog:
    def __init__(self, v):
        self.v = v
    def __eq__(self, other):
        return self.v == other.v
    def __hash__(self):
        return id(self)

try: 
    kennel = {Dog(0):"spunky", Dog(0):"needs shots", Dog(0):"pickup tomorrow"}
except Exception as e:
    print(e)
    
print(kennel)

{<__main__.Dog object at 0x10e8130f0>: 'spunky', <__main__.Dog object at 0x10e813080>: 'needs shots', <__main__.Dog object at 0x10e813128>: 'pickup tomorrow'}


We're back in the money.  `__hash__` has given us a safe way in which to determine inequality.

In [13]:
dog1, dog2 = Dog(1), Dog(2)
dog1.v = dog2.v = 3  # force same v
print(dog1 == dog2)

True


And yet:

In [14]:
{dog1:"dog1", dog2:"dog2"}  # hashability is restored

{<__main__.Dog at 0x10e8137f0>: 'dog1', <__main__.Dog at 0x10e813828>: 'dog2'}

In [15]:
print({dog1:"dog1", dog2:"dog2"}[dog1])
print({dog1:"dog1", dog2:"dog2"}[dog2])

dog1
dog2


An evil thought may occur to one:  might we deliberately define ```__hash__``` in such a way that corruption could be possible?  Lets try:

In [16]:
class Dog:
    def __init__(self, v):
        self.v = v
    def __eq__(self, other):
        return self.v == other.v
    def __hash__(self):
        return True

try:
    dog1 = Dog(3)
    dog2 = Dog(4)
    kennel = {dog1:"spunky", dog2:"needs shots"}
except Exception as e:
    print(e)

In [17]:
dog1, dog2 = Dog(3), Dog(4)
dog1.v = dog2.v = 3  # force same v
print(dog1 == dog2)
print({dog1:"dog1", dog2:"dog2"}[dog1])
print({dog1:"dog1", dog2:"dog2"}[dog2])

True
dog2
dog2


In [18]:
kennel

{<__main__.Dog at 0x10e802470>: 'needs shots',
 <__main__.Dog at 0x10e8024a8>: 'spunky'}

In [19]:
# CASE 1
print("CASE 1")
print("Same value?:", dog1 == dog2)
print("kennel[dog1]):", kennel[dog1])
print("kennel[dog2]):", kennel[dog2])

# CASE 2
print("CASE 2")
dog1.v = dog2.v
print("Same value?:", dog1 == dog2)
print("kennel[dog1]):", kennel[dog1])
print("kennel[dog2]):", kennel[dog2])

# CASE 3
print("CASE 3")
dog2.v = 4
print("Same value?:", dog1 == dog2)
print("kennel[dog1]):", kennel[dog1])
print("kennel[dog2]):", kennel[dog2])

CASE 1
Same value?: True
kennel[dog1]): spunky
kennel[dog2]): spunky
CASE 2
Same value?: True
kennel[dog1]): spunky
kennel[dog2]): spunky
CASE 3
Same value?: False
kennel[dog1]): spunky
kennel[dog2]): needs shots


## Callable versus Not Callable

Another distinction we spent some time on:  the difference between objects it makes sense to "call", versus those considered "uncallable".  

```obj( )``` is the syntax in question, i.e. parentheses attached to an object like a mouth, into which arguments may be inserted, to be matched with parameters in the callable's definition.

In other words, regardless of whether arguments are appropriate, any object that performs work, raises no exception, when "given a mouth", is what we call a "callable".  

The function callable, a built-in, returns True or False with respect to that question.

In [20]:
callable(3)

False

In [21]:
callable(callable)

True

In [22]:
callable(True)

False

In [23]:
def uncallable(obj):
    return not callable(obj)

In [24]:
uncallable(3)

True

In [25]:
uncallable(uncallable)

False

If one is a callable, then one has a ```__call__``` method, i.e. the following should be a truism:

In [26]:
for obj in ([], 3, {}, (lambda x: x)):
    print(callable(obj) == ("__call__" in dir(obj)))

True
True
True
True


## Iterable Versus Iterator

An iterable is any object it makes sense to for loop over, such as collections, but not individual integers.  The iterator is the object actually doing the iterating.

Think of an iterable as a sequence of lilly pads across a pond, with the iterators being the frogs that hop along them, from one to the other.

The way to check if an object is an iterator is to look for both ```__next__``` and ```__iter__```.  Those are the spots that tell you it's a leopard.  We often classify classes according to their signature methods.

An iterable needs at least a ```__getitem__```, in which case Python's built-in iter( ) will assume consecutive integers starting from 0, fed to ```__getitem__```, is a sensible meaning for the iterator's ```__next__```.

In [27]:
class Thing:
    
    def __init__(self, L):
        self.__L = L
        
    def __iter__(self):
        # generator expression
        return (it for it in reversed(self.__L))
    
thething = Thing([1,2,3,4,5,6]) # initialize the iterable

for element in thething:  # iter(thething) is implicitly applied
    print(element, end= " ")


6 5 4 3 2 1 

## Generator, Descriptor and Context Manager

Now that we've identified the iterator by its methods, we might do so with respect to these other categories:  generator, descriptor and context manager.

Let's go a little deeper with the generator.

The generator is really a subcategory of iterator in that it has both ```__iter__``` and ```__next__```.  What it will also have, though, is a ```send``` method, which indicates either the presence of a ```yield``` in the source code, or a generator expression.  

Iterators implemented using ```yield``` in principle have the capability receive through ```send``` as well, although this capability may not be implemented.

In [28]:
the_iter = iter([]) # iterator in general
print("Generator?:", "send" in " ".join(dir(the_iter)))

Generator?: False


In [29]:
L = list(range(30))
g = (it for it in reversed(L))
print(next(g))
print("Generator?:", "send" in " ".join(dir(g)))

29
Generator?: True


In [30]:
def Tractor():
    n = 0
    tank = 10
    while tank:
        gas = (yield n)
        if gas:
            tank += gas
            print("\ntank at", tank)
        else:
            n += 1
            tank -= 1
    return "Gold & Silver"
        
t = Tractor()
for pos in t:
    print(pos, end=" ")

0 1 2 3 4 5 6 7 8 9 

In [31]:
t = Tractor()
while True:
    try:
        pos = next(t)
    except StopIteration:
        break
    else:
        print(pos, end=" ")
        if pos == 8:  # time for refill
            t.send(8)
            print("kaching!")

0 1 2 3 4 5 6 7 8 
tank at 10
kaching!
9 10 11 12 13 14 15 16 17 

In [32]:
def driver():
    yield "Hello, I'm going to delegate to another generator\n"
    t = Tractor()
    result = (yield from t)
    yield "\nI have finished delegating and now will throw away the result"
    
gen = driver()
for s in gen:
    print(s, end=" ")

Hello, I'm going to delegate to another generator
 0 1 2 3 4 5 6 7 8 9 
I have finished delegating and now will throw away the result 

Recovering a ```return``` object from a generator is no mean trick.  Since ```yield``` is doing what ```return``` used to do, we're actually, in providing a `return` statement, effectively raising a StopIteration. 

We might choose to see this raised exception as a shutting down from within, a turning off the lights.  

Yet there's an opportunity to leave a message in a bottle, to be discovered posthumously, so to speak.  Buried in the "wreckage" of a StopIteration is its value.

The trick is to not let a for loop "smooth over" the internal shutdown, nor the ```yield from```.  

In the latter case, having a name ready to catch this value, is a matter of simple assignment.
```python
    result = (yield from t)
```

However such syntax makes sense already in a generator.  The final return of this result outside any generator context, will need to be done during exception handling, as demonstrated below:

In [33]:
def driver():
    yield "Hello, I'm going to delegate to another generator\n"
    t = Tractor()
    result = (yield from t)
    yield "\nI have finished delegating and now will keep the result\n"
    return result

gen = driver()
while True:
    try:
        print(next(gen), end=" ")
    except StopIteration as e:
        print(e.value)
        break

Hello, I'm going to delegate to another generator
 0 1 2 3 4 5 6 7 8 9 
I have finished delegating and now will keep the result
 Gold & Silver


`Gold & Silver` is what the Tractor instance returns when it runs out of gas.  


Why a tractor would return such a string doesn't make a lot of sense, unless we're talking about the gold and silver it might take to effect repairs.  

The idea is to suggest something valuable we really want to recover, and because we go to the trouble, in a `while True:` loop, to push though to a `StopIteration` we then handle explicitly, our mission is thereby accomplished.